In [ ]:
#default_exp random_genome_to_midi

In [ ]:
#export
import mido
import time
import random
from dataclasses import dataclass

In [ ]:
outport = mido.open_output('mido_out', virtual=True)

In [ ]:
outport.send(mido.Message('note_on', note=60))

In [ ]:
outport.send(mido.Message('note_off', note=60))

In [ ]:
#export
def random_genome(length):
    alphabet = 'UDFBS'
    return ''.join([random.sample(alphabet, k=1)[0] for _ in range(length)])

In [ ]:
random_genome(10)

In [ ]:
#export
@dataclass
class Note:
    pitch: int
    begin: float
    length: float

In [ ]:
#export
def events_length(genome):
    numeric = lambda g: {'U':1, 'D':2, 'F':3, 'B':5, 'S':7}[g]

    output = []
    t_pos_t = 0
    t_pos_p = 60
    t_len = 1
    saw_S = False
    
    c = iter(genome)

    try:
        while True:
            l = next(c)
            if l == 'S':
                if saw_S:
                    # second S, undo stamp and resize turtle
                    del output[-1]
                    t_len *= numeric(next(c)) / numeric(next(c))
                else:
                    saw_S = True
                    output.append(Note(t_pos_p, t_pos_t, t_len))
                    continue

            elif l == 'U':
                t_pos_p += 1
            elif l == 'D':
                t_pos_p -= 1
            elif l == 'F':
                t_pos_t += t_len
            elif l == 'B':
                t_pos_t -= t_len
            else:
                raise ValueError(f'Unexpected letter in genome: {l}')

            saw_S = False

    except StopIteration:
        return output

In [ ]:
events_length(random_genome(40))

In [ ]:
#export
@dataclass
class NoteOn:
    pitch: int
    time: float
        
@dataclass
class NoteOff:
    pitch: int
    time: float

In [ ]:
#export
def events_on_off(events):
    output = []
    for e in events:
        output.append(NoteOn(e.pitch, e.begin))
        output.append(NoteOff(e.pitch, e.begin+e.length))
        
    return sorted(output, key=lambda e: e.time - (1e-9 if isinstance(e,NoteOff) else 0))

In [ ]:
l = events_on_off(events_length(random_genome(40)))
l

In [ ]:
#export
def genome2midi(genome):
    midifile = mido.MidiFile(type=0)
    track = mido.MidiTrack()
    midifile.tracks.append(track)
    
    ppq = midifile.ticks_per_beat

    events = iter(events_on_off(events_length(genome)))

    e = None
    try:
        n = next(events)
        while True:
            e = mido.Message('note_on' if isinstance(n, NoteOn) else 'note_off', note=n.pitch)
            e_start = n.time
            n = next(events)
            e.time = round((n.time - e_start) * ppq)
            track.append(e)
        
    except StopIteration:
        if e is not None:
            track.append(e)
            
    return midifile

In [ ]:
song = genome2midi(random_genome(80))
song

In [ ]:
for m in song.play():
    outport.send(m)